**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/group-by-having-count).**

---


# Introduction

Queries with **GROUP BY** can be powerful. There are many small things that can trip you up (like the order of the clauses), but it will start to feel natural once you've done it a few times. Here, you'll write queries using **GROUP BY** to answer questions from the Hacker News dataset.

Before you get started, run the following cell to set everything up:

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex3 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


The code cell below fetches the `comments` table from the `hacker_news` dataset.  We also preview the first five rows of the table.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=25).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0
5,3867418,Iv,Iv,1334921984,2012-04-20 11:39:44+00:00,&#62; There's a whole class of 'mom &#38; pop'...,3867404,None,None,0
6,3925617,Iv,Iv,1336076765,2012-05-03 20:26:05+00:00,I'm also in this ballpark (300-600€ / day) whi...,3924840,None,None,0
7,3107534,Iv,Iv,1318520044,2011-10-13 15:34:04+00:00,how do you run unity in non-accelerated mode ?...,3107241,None,None,0
8,8409259,Iv,Iv,1412421647,2014-10-04 11:20:47+00:00,Polio is not exterminated even if it is absent...,8409226,None,None,0
9,2855741,Jd,Jd,1312690646,2011-08-07 04:17:26+00:00,"Yep, I didn't find Rescuetime very helpful. I ...",2855343,None,None,0


# Exercises

### 1) Prolific commenters

Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts `NumPosts`.

In case sample query is helpful, here is a query you saw in the tutorial to answer a similar question:
```
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """
```

In [3]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
        SELECT author, COUNT(id) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY author
        HAVING COUNT(id) > 10000
        """
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

# Check your answer
q_1.check()

         author  NumPosts
0  dragonwriter     10723
1          None    227736
2           eru     10448
3       rbanffy     10557
4         DanBC     12902


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [4]:
#q_1.solution()

### 2) Deleted comments

How many comments have been deleted? (If a comment was deleted, the `deleted` column in the comments table will have the value `True`.)

In [5]:
# Write your query here and figure out the answer

In [6]:
num_deleted_posts = """
        SELECT COUNT(id) AS NumDeletedPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY deleted
        HAVING deleted=True
        """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(num_deleted_posts, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
num_deleted_posts = query_job.to_dataframe()

# View top few rows of results
num_deleted_posts=num_deleted_posts['NumDeletedPosts'].values[0]
# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [7]:
#q_2.solution()

# Keep Going
**[Click here](https://www.kaggle.com/dansbecker/order-by)** to move on and learn about the **ORDER BY** clause.

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*